# Allocating Qubits on QPU Devices

This notebook demonstrates how you can specify explicitly which qubits to use when you run a quantum circuit on QPU devices from Rigetti.

When you submit a circuit for execution on a QPU, Amazon Braket performs a series of compilation steps: it maps the _abstract qubits_ in your circuit to _physical qubits_ in the device; it synthesizes gates into the native gate set of the device; it optimizes the circuit to reduce the number of gates; and finally, it translates the gates into executable pulses.

This section shows how the first step, called qubit allocation, works for the Rigetti Aspen-8 device.

In [1]:
# general imports
from braket.aws import AwsDevice
from braket.circuits import Circuit
import numpy as np

__NOTE__: Enter your desired device and S3 location (bucket and prefix). Remember that bucket names for Amazon Braket always begin with "amazon-braket-". 

In [2]:
# enter the S3 bucket you created during onboarding in the code as follows
my_bucket = "amazon-braket-Your-Bucket-Name" # the name of the bucket
my_prefix = "Your-Folder-Name" # the name of the folder in the bucket

s3_folder = (my_bucket, my_prefix)

## Automatic qubit allocation

Qubit allocation for Rigetti devices on Amazon Braket utilizes [the Quil Compilers](https://pyquil-docs.rigetti.com/en/latest/compiler.html#the-quil-compiler)'s _rewiring_ strategies. By default, when you submit a circuit on Amazon Braket to a Rigetti device, the circuit is rewired according to the [PARTIAL](https://pyquil-docs.rigetti.com/en/latest/compiler.html#partial) rewiring strategy. Specifically, the compiler starts with an empty mapping from logical to physical qubits. Taking into account the latest calibration data of the device, the compiler fills in the mapping with the goal, sequentially, to maximize the overall fidelity of the circuit.

The example that follows shows how to create a GHZ state on qubits that are not physically connected. After the task is completed, you can obtain a list of the actual gates executed on the device, by viewing the result metadata.

First, instantiate the Rigetti Aspen-8 device and retrieve its connectivity graph, which shows the qubits that are directly connected on the chip.

In [3]:
device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-8")

connectivity_graph = device.properties.paradigm.connectivity.connectivityGraph
print(f"the connectivity of {device.name} is: {connectivity_graph}")

the connectivity of Aspen-8 is: {'0': ['7'], '1': ['2', '16'], '2': ['1', '3', '15'], '3': ['2', '4'], '4': ['3', '5'], '5': ['4', '6'], '6': ['5', '7'], '7': ['0', '6'], '10': ['11', '17'], '11': ['10', '12', '26'], '12': ['11', '13', '25'], '13': ['12'], '15': ['2', '16'], '16': ['1', '15', '17'], '17': ['10', '16'], '20': ['21', '27'], '21': ['20', '22', '36'], '22': ['21', '23', '35'], '23': ['22', '24'], '24': ['23', '25'], '25': ['12', '24', '26'], '26': ['11', '25', '27'], '27': ['20', '26'], '30': ['31', '37'], '31': ['30', '32'], '32': ['31', '33'], '33': ['32', '34'], '34': ['33', '35'], '35': ['22', '34', '36'], '36': ['21', '35', '37'], '37': ['30', '36']}


Next, create a GHZ circuit with three qubits 0, 2, 4, and run it on the Aspen-8 device. Notice that none of these qubits are connected on the Aspen-8 connectivity graph.

In [4]:
# create a GHZ state with non-neighboring qubits
circuit = Circuit()
circuit.h(0).cnot(0,2).cnot(0,4)
print(circuit)

rigetti_rewiring = device.run(circuit, s3_folder, shots=100)

T  : |0|1|2|
            
q0 : -H-C-C-
        | | 
q2 : ---X-|-
          | 
q4 : -----X-

T  : |0|1|2|


In [6]:
print("Status of task:", rigetti_rewiring.state())

Status of task: COMPLETED


To verify the final qubit allocation, retrieve the compiled program that was executed:

In [7]:
result = rigetti_rewiring.result()
counts = result.measurement_counts
print("Measurement counts:", counts)
print("The compiled circuit is:\n", result.additional_metadata.rigettiMetadata.compiledProgram)

Measurement counts: Counter({'111': 27, '000': 16, '110': 13, '010': 13, '100': 9, '101': 9, '001': 7, '011': 6})
The compiled circuit is:
 DECLARE ro BIT[3]
PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
RZ(pi) 15
RZ(-pi/2) 16
RX(-pi/2) 16
XY(pi) 16 15
RZ(-pi/2) 16
RX(-pi/2) 16
RZ(pi/2) 16
XY(pi) 16 15
RZ(-pi/2) 1
RX(pi/2) 1
CZ 1 16
RX(-pi/2) 1
RZ(pi/2) 1
RX(-pi/2) 15
RZ(pi) 16
MEASURE 1 ro[2]
MEASURE 15 ro[1]
MEASURE 16 ro[0]



Notice that the PARTIAL rewiring was applied. The qubits 0, 2, 4 in the original circuit were mapped to the qubits 16, 15, 1 in the Rigetti device, and the gates were compiled into native gates.

## User-defined qubit allocation

In Amazon Braket, you can choose to prescribe a qubit mapping manually, and prevent further rewiring for Rigetti devices. To enable manual mapping, set `disable_qubit_rewiring=True` when submitting the task to run.

If all the gates in the circuit satisfy the topological constraints of the device, Amazon Braket maps abstract qubit $i$ in the circuit to the physical qubit $i$ in the device, and maps qubit pair $(i, j)$ to the connection $(i, j)$ in the device. On the other hand, Amazon Braket raises an exception if a specified qubit or qubit pair do not exist in the device connectivity graph.

In [8]:
# create a random state with neighboring qubits
circuit = Circuit()
circuit.rz(0,np.pi/2).cnot(1,2).x(3)
print(circuit)
rigetti_task = device.run(circuit, s3_folder, shots=100, disable_qubit_rewiring=True)

T  : |   0    |
               
q0 : -Rz(1.57)-
               
q1 : -C--------
      |        
q2 : -X--------
               
q3 : -X--------

T  : |   0    |


In [9]:
print("Status of task:", rigetti_task.state())

Status of task: COMPLETED


In [10]:
result = rigetti_task.result()
counts = result.measurement_counts
print("Measurement counts:", counts)
print("The compiled circuit is:\n", result.additional_metadata.rigettiMetadata.compiledProgram)

Measurement counts: Counter({'0001': 48, '0101': 20, '0111': 14, '0000': 5, '1001': 3, '0011': 3, '0110': 3, '0100': 2, '1000': 1, '0010': 1})
The compiled circuit is:
 DECLARE ro BIT[4]
PRAGMA INITIAL_REWIRING "NAIVE"
RESET
RZ(pi/2) 1
RZ(pi) 2
XY(pi) 1 2
RZ(-pi/2) 1
RX(-pi/2) 1
RZ(pi/2) 1
XY(pi) 1 2
RZ(pi/2) 0
RX(-pi/2) 2
RX(pi) 3
MEASURE 3 ro[3]
MEASURE 2 ro[2]
MEASURE 1 ro[1]
MEASURE 0 ro[0]



The qubits 0, 1, 2, 3 in the original circuit were mapped to the physical qubits 0, 1, 2, 3 in the device. Other compilation steps, such as gate synthesis and circuit optimization, are still performed. These steps allow the circuit to run successfully and improve the overall fidelity.

## Devices that do not support allocation

The IonQ device does not support manual allocation. For circuits submitted to the IonQ device, qubits are allocated automatically.

In [23]:
circuit = Circuit()
circuit.h(0).cnot(0,2).rz(1,1.2)

device = AwsDevice("arn:aws:braket:::device/qpu/ionq/ionQdevice")
ionq_task = device.run(circuit, s3_folder, shots=100)

In [26]:
print("Status of task:", ionq_task.state())

Status of task: COMPLETED


In [29]:
result = ionq_task.result()
counts = result.measurement_counts
print("Measurement counts:", counts)

Measurement counts: Counter({'101': 57, '000': 40, '100': 1, '110': 1, '001': 1})
